$\textbf{IMPORTS}$

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$

In [99]:
#1.Load pages boliga.dk
links = []
for page in range(1,147):
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{SCRAPE DATA FROM EACH PAGE}, BOLIGA.DK$

In [100]:
#Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

for i in links:
    response = requests.get(i)
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(',',1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
    #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"

$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [103]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Price": price_final, "Address": address_final, "Zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data = pd.DataFrame(df_addzip)


In [104]:
df_data

,Price,Address,Zipcode,m2,rooms
0,595.000kr.,Grøfthøjparken 151,8260 Viby J,33,1
1,995.000kr.,Stranden 70,9560 Hadsund,66,4
2,1.095.000kr.,Odinsvej 10,4700 Næstved,72,2
3,1.198.000kr.,Strubjerg 128,9400 Nørresundby,82,2
4,1.248.000kr.,Ingemanns Alle 175B,6700 Esbjerg,72,2
...,...,...,...,...,...
7166,600.000kr.,Chr. Møllers Vej 6D,9990 Skagen,72,3
7167,390.000kr.,Ringgade 82,7600 Struer,66,2
7168,595.000kr.,Tranevej 108,9990 Skagen,74,3
7169,345.000kr.,Fjelstervangvej 26A,6933 Kibæk,82,4
